# **Aprendizaje automático en Python**
El aprendizaje automático (ML, por sus siglas en inglés) y la inteligencia artificial (IA) son aspectos increíblemente importantes del procesamiento del lenguaje en la actualidad. A menudo, los idiomas bien estudiados y documentados, como el inglés y el español, tienen enormes modelos de aprendizaje automático, entrenados sobre miles de millones y miles de millones de palabras. Sin embargo, también podemos usar el aprendizaje automático, a una escala menor, para idiomas poco estudiada y documentadas.

Esta lección te dará una breve visión general del uso de un framework de aprendizaje automático en Python, pero no entrará en una gran profundidad de la discusión. Para aprender más sobre cómo usar el aprendizaje automático para el procesamiento de lenguas, recomendamos el libro [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) de Jurafsky y Martin.

## **Análisis de Sentimento**
La tarea que resolveremos con un modelo ML es el **análisis de sentimientos**. El análisis de sentimiento tiene como objetivo predecir si una porción de texto expresa un sentimiento positivo, negativo o neutral sobre el tema. Utilizaremos un conjunto de tweets de [here](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset?resource=download). Por ejemplo, el siguiente tweet se clasifica como positivo:

> Al diablo con las críticas, Wolverine me pareció increíble. Pero no hay suficiente Dominic Monaghan para mi gusto. Pero no hay suficiente Dominic Monaghan para mi gusto.

Pero este tweet es negativo:

> ¡¡¡ESTE twitter me está volviendo loco...NO ME DEJA DESCARGAR UNA FOTO DE PERFIL!!! ¡¡...supongo que seguiré intentándolo!!

## **Cargando datos**
Primero, necesitamos cargar nuestro conjunto de datos y prepararlo para usarlo en un modelo. Dado que los datos están en un formato csv, debemos usar el módulo `csv` para ayudarnos a analizarlos.

In [ ]:
import csv

all_tweets = []
all_sentiments = []

with open('./Tweets.csv') as tweets_file:
    # Create a csv parser
    csv_reader = csv.reader(tweets_file)
    
    # Skip the first row, the headers
    next(csv_reader, None)
    
    for row in csv_reader:
        # The second column has the tweet text
        all_tweets.append(row[1])
        
        # The fourth column has the sentiment label
        all_sentiments.append(row[3])
        
print(all_tweets[:10])
print(all_sentiments[:10])

También reemplazaremos cada sentimiento por una etiqueta: 0 para neutral, 1 para positivo y 2 para negativo.

In [ ]:
all_sentiments_encoded = []
for sentiment in all_sentiments:
    if sentiment == 'neutral':
        all_sentiments_encoded.append(0)
    elif sentiment == 'positive':
        all_sentiments_encoded.append(1)
    elif sentiment == 'negative':
        all_sentiments_encoded.append(2)
    else:
        print("Unexpected label found")
        break
        
print(all_sentiments_encoded[:10])

### División de entrenamiento/pruebas
En el aprendizaje de máquinas, es usual dividir los datos en dos conjuntos: uno para entrenar el modelo y otro para probar el rendimiento posterior del modelo. Esto ayuda a evaluar el modelo de forma justa y a evitar el *sobreajustamiento*, a saber, que el modelo sólo funcione bien con los datos de entrenamiento.

Utilizaremos el método `train_test_split` del paquete `sklearn`.

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_labels, test_labels = train_test_split(all_tweets, all_sentiments_encoded, test_size=0.3)

print(len(train_sentences), "training sentences")
print(len(test_sentences), "testing sentences")

## **Creando un Modelo**
Ahora estamos listos para crear nuestro modelo. Utilizaremos [Keras](https://keras.io), uno de los frameworks más populares para el aprendizaje automático. Keras ofrece la configuración más sencilla pero la menos personalizable, por lo que es una buena opción para esta lección.

### Vectorización
Primero, nuestro modelo convertirá cada frase en un vector de valores binarios, donde cada posición representa la ocurrencia de una palabra. Por ejemplo, si el vector para una oración comienza con `[1, 0, ...]` y las palabras son `[malo, bueno, .. ]`, entonces el vector indica que la palabra `mal` ocurre en el tweet, pero la palabra `bueno` no.

Para esto, usamos la capa keras `TextVectorization`.

In [ ]:
from tensorflow import keras

text_vectorizer = keras.layers.TextVectorization(output_mode='multi_hot', # Create a vector in the style we described
                                                 max_tokens=2500)         # Use only the 2500 most common words

# Train the vectorizer using the training dataset
text_vectorizer.adapt(train_sentences)

Ahora podemos ver las 100 palabras más comunes en el conjunto de datos.

In [ ]:
print(text_vectorizer.get_vocabulary()[:100])

También podemos usar el vectorizador para codificar una frase y ver cómo se ve el resultado:

In [ ]:
text_vectorizer("I went to the store")

### Capas ocultas
<div>
<img src="../assets/mlp.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

Una de las técnicas clave utilizadas en el aprendizaje automático es la incorporación de **capas ocultas**. En cada capa oculta, una función se aplica a las entradas con variables de peso que modifican la salida. El modelo ajusta las variables de peso durante el entrenamiento hasta que se prevea la salida correcta.

Utilizar más capas ocultas permite que el modelo aprenda patrones más complicados. En este caso, los pesos determinarán cuánto contribuye cada palabra a la predicción final. 

In [ ]:
# The parameter indicates how many nodes will be in each layer
hidden_layer1 = keras.layers.Dense(100)

### Poniéndolo todo junto 
Ahora, podemos reunir todos los componentes de nuestro modelo.

Cada modelo debe usar una **función de pérdida**, que define cuánto error hay. El modelo intentará minimizar la función de pérdida y así hacer mejores predicciones. En este caso, utilizamos **pérdidas de entropía cruzada ('crossentropy loss')**, que calcula cuánto error hubo en una predicción entre etiquetas categóricas.

In [ ]:
model = keras.Sequential()

# Our inputs will be strings
input_layer = keras.Input(shape=(1,), dtype='string')
model.add(input_layer)

# Add the vectorization layer
model.add(text_vectorizer)

# Add the hidden layers
model.add(hidden_layer1)

# Add the output layer
# Since we have 3 possible output classes, the layer should have three nodes
output_layer = keras.layers.Dense(3, activation='softmax')
model.add(output_layer)

# Compile the model
# We use `categorical_crossentropy` for tasks that have multiple categorical outputs
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

### Entrenamiento
Ahora que hemos construido un modelo, el siguiente paso es entrenarlo. Esto puede llevar algún tiempo, ya que el entrenamiento implica muchas operaciones matriciales de gran tamaño.

In [ ]:
history = model.fit(train_sentences, train_labels, epochs=7, verbose=True)

Podemos ver que a medida que nuestra pérdida disminuyó, la precisión de la formación aumentó. 

### Evaluación
Ahora, vamos a evaluar el modelo con nuestros datos de prueba.

In [ ]:
model.evaluate(test_sentences, test_labels)

Nuestra exactitud de la prueba fue algo menor que nuestra precisión en el entrenamiento, pero todavía mucho mejor que adivinar al azar. Crear modelos en los que el rendimiento de la prueba no sea significativamente peor que el rendimiento del entrenamiento es un objetivo clave en el aprendizaje automático.

Por último, veamos nuestro modelo en acción. Podemos utilizar nuestro modelo para predecir el sentimiento de algún Tweet que inventemos.

In [ ]:
import numpy as np

def predict_sentiment(tweet):
    # Our predictions will be a 3-element vector, where each element is the probability of a given sentiment class
    predictions = model.predict([tweet])[0]
    
    # Take the argmax to find the most likely sentiment
    predicted_sentiment_index = np.argmax(predictions)
    
    # Turn the sentiment index into a label
    all_sentiments = ['neutral', 'positive', 'negative']
    predicted_sentiment = all_sentiments[predicted_sentiment_index]
    
    return predicted_sentiment
    

print(predict_sentiment("I loved the new Guardians of the Galaxy movie. It was so well-made and touching!"))
print(predict_sentiment("I hated that movie. Gunn is a talentless hack"))

## **Ejercicio de desafío 1**
Intente modificar el modelo utilizado aquí para mejorar el rendimiento. Experimenta con el uso de un vocabulario más grande en el `TextVectorizer`, usando un número diferente de capas ocultas, o capas ocultas con un número diferente de nodos.

In [ ]:
# TODO: Build and train a modified model


## **Ejercicio de desafío 2**
Descargar [este conjunto de datos](https://www.kaggle.com/datasets/azimulh/tweets-data-for-authorship-attribution-modelling). Crea y entrena un modelo para predecir el autor de un tweet. Se trata de un problema similar al análisis de sentimientos, excepto que tenemos más de 3 etiquetas posibles.

In [ ]:
# TODO: Build and train a model for authorship prediction


## **Conclusión**
En esta lección, hemos aprendido cómo crear, entrenar y evaluar un modelo de aprendizaje automático en Python para tareas de procesamiento del lenguaje. 
- Crear representaciones vectoriales de texto usando `TextVectorizer`
- Construir un modelo con capas ocultas
- Entrenar un modelo con un conjunto de datos de entrenamiento
- Evaluar un modelo con un conjunto de datos de prueba

El aprendizaje automático puede ser una herramienta poderosa para las aplicaciones del lenguaje, y puede aplicarse a una amplia gama de tareas. Independientemente de la tarea, las técnicas básicas mostradas aquí se utilizarán una y otra vez. 

En este punto, has completado todas las habilidades necesarias para comenzar a construir una tecnología de lenguaje útil. A continuación, echa un vistazo a los proyectos!